In [2]:
# 📚 Import librerie necessarie
import pandas as pd

# 📂 Percorso file
file_path = r"C:\Users\Lorenzo\Desktop\scoring/sleep_characteristics.xlsx"

# 📥 Caricamento dati
df = pd.read_excel(file_path)

# 🔍 Visualizziamo le prime righe del dataset
df.head()


,Patient,Group,Sleep Efficiency (%),Total Sleep Time (min),N1%,N2%,N3%,REM%,N1 Time (min),N2 Time (min),N3 Time (min),REM Time (min),WASO (min)
0,PD009,CTL,62.000000,310.0,8.600000,32.500000,12.600000,8.300000,43.0,162.5,63.0,41.5,190.0
1,PD010,CTL,63.757576,263.0,9.090909,33.212121,14.424242,7.030303,37.5,137.0,59.5,29.0,149.5
2,PD020,CTL,79.487179,356.5,16.387960,38.127090,12.709030,12.263099,73.5,171.0,57.0,55.0,92.0
3,PD022,CTL,39.261745,175.5,11.968680,20.693512,6.599553,0.000000,53.5,92.5,29.5,0.0,271.5
4,PD025,CTL,94.029851,315.0,8.507463,52.388060,10.746269,22.388060,28.5,175.5,36.0,75.0,20.0


In [3]:
# 📌 Stampiamo le informazioni sul dataset
df.info()

# 📊 Contiamo i valori mancanti
df.isnull().sum()

# 📋 Stampiamo i nomi di tutte le colonne
print("\n📌 Colonne disponibili nel dataset:")
print(df.columns.tolist())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Patient                 30 non-null     object 
 1   Group                   30 non-null     object 
 2   Sleep Efficiency (%)    30 non-null     float64
 3   Total Sleep Time (min)  30 non-null     float64
 4   N1%                     30 non-null     float64
 5   N2%                     30 non-null     float64
 6   N3%                     30 non-null     float64
 7   REM%                    30 non-null     float64
 8   N1 Time (min)           30 non-null     float64
 9   N2 Time (min)           30 non-null     float64
 10  N3 Time (min)           30 non-null     float64
 11  REM Time (min)          30 non-null     float64
 12  WASO (min)              30 non-null     float64
dtypes: float64(11), object(2)
memory usage: 3.2+ KB

📌 Colonne disponibili nel dataset:
['Patient', '

In [4]:
from scipy.stats import skew, kurtosis

# 📌 Lista delle colonne numeriche
numerical_cols = df.columns[2:]  # Esclude 'Patient' e 'Group'

# 📊 Creazione del riassunto statistico per ogni gruppo
summary_stats = df.groupby("Group")[numerical_cols].agg(["mean", "median", "var", skew, kurtosis])

# 📄 Visualizzazione
summary_stats


Sleep Efficiency (%)                                             \
                      mean     median         var      skew  kurtosis   
Group                                                                   
ADV              67.442680  65.325861   63.492682  1.462899  1.437510   
CTL              70.527403  73.793860  309.463483 -0.545161 -0.384024   
DNV              75.596499  74.858437   51.987391 -0.102017 -1.268776   
DYS              63.252448  58.165081  239.602001  0.227143 -1.733611   

      Total Sleep Time (min)                                           ...  \
                        mean  median          var      skew  kurtosis  ...   
Group                                                                  ...   
ADV                  329.375  347.25  2419.410714 -1.026777 -0.052900  ...   
CTL                  303.000  315.00  4302.666667 -1.091524  0.052284  ...   
DNV                  349.500  366.00   588.375000 -0.529943 -1.563794  ...   
DYS                  296.550  270.00  7858.636111  0.247342 -1.477486  ...   

      REM Time (min)                                          WASO (min)  \
                mean median          var      skew  kurtosis        mean   
Group                                                                      
ADV        67.125000  61.25  1101.267857  0.372550 -1.028401  161.062500   
CTL        45.928571  43.50   724.285714 -0.416062 -0.684552  132.285714   
DNV        86.200000  85.50   584.950000  0.235710 -1.431454  114.400000   
DYS        36.150000  25.75  1002.891667  0.533611 -1.160724  167.850000   

                                                
       median          var      skew  kurtosis  
Group                                           
ADV    165.00  2345.245536 -0.942108  0.164947  
CTL    119.50  6630.404762  0.432170 -0.549311  
DNV    111.00  1505.175000  0.094133 -1.454466  
DYS    184.75  4682.113889  0.068322 -1.359867  

[4 rows x 55 columns]

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import levene, fligner

# 📂 Percorso del file
file_path = r"C:\Users\Lorenzo\Desktop\scoring/sleep_characteristics.xlsx"

# 📊 Caricamento del dataset
df = pd.read_excel(file_path)

# 📌 Selezione delle variabili continue (escludiamo 'Patient' e 'Group')
numerical_cols = df.columns[2:]

# 📌 Gruppi di interesse
group_order = ["CTL", "DNV", "ADV", "DYS"]

# 📁 Cartella per salvare i risultati
output_dir = os.path.dirname(file_path)
qq_plot_dir = os.path.join(output_dir, "qq_plots")
os.makedirs(qq_plot_dir, exist_ok=True)


In [6]:
for col in numerical_cols:
    plt.figure(figsize=(12, 8))

    for i, group in enumerate(group_order, 1):
        data = df[df["Group"] == group][col].dropna()

        if len(data) > 3:  # Il test richiede almeno 3 dati
            plt.subplot(2, 2, i)
            stats.probplot(data, dist="norm", plot=plt)
            plt.title(f"Q-Q Plot di {col} - {group}")
            plt.xlabel("Quantili teorici")
            plt.ylabel("Quantili campionari")

    plt.tight_layout()

    # 📂 Salvataggio del grafico
    plot_path = os.path.join(qq_plot_dir, f"QQPlot_{col.replace(' ', '_')}.png")
    plt.savefig(plot_path, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"📊 Salvato: {plot_path}")

print("✅ Tutti i Q-Q plot sono stati generati e salvati!")


📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_Sleep_Efficiency_(%).png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_Total_Sleep_Time_(min).png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_N1%.png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_N2%.png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_N3%.png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_REM%.png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_N1_Time_(min).png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_N2_Time_(min).png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_N3_Time_(min).png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_REM_Time_(min).png
📊 Salvato: C:\Users\Lorenzo\Desktop\scoring\qq_plots\QQPlot_WASO_(min).png
✅ Tutti i Q-Q plot sono stati generati e salvati!


In [ ]:
# 📂 Caricamento risultati Shapiro-Wilk per verificare normalità
shapiro_df = pd.read_excel(shapiro_excel_path)

levene_results = []
fligner_results = []

for col in numerical_cols:
    group_data = [df[df["Group"] == g][col].dropna() for g in group_order]

    # **Levene’s Test**
    stat_levene, p_levene = levene(*group_data)
    levene_results.append({"Variabile": col, "Levene’s p-value": p_levene})

    # **Fligner-Killeen Test SOLO se almeno un gruppo è non normale**
    normality_check = shapiro_df[shapiro_df["Variabile"] == col]["Shapiro-Wilk p-value"] < 0.05
    if normality_check.any():
        stat_fligner, p_fligner = fligner(*group_data)
        fligner_results.append({"Variabile": col, "Fligner-Killeen p-value": p_fligner})

# 📊 Creazione DataFrame risultati
levene_df = pd.DataFrame(levene_results)
fligner_df = pd.DataFrame(fligner_results)

# 📂 Salvataggio in Excel
levene_path = os.path.join(output_dir, "levene_results.xlsx")
fligner_path = os.path.join(output_dir, "fligner_results.xlsx")

levene_df.to_excel(levene_path, index=False)
fligner_df.to_excel(fligner_path, index=False)

# 📋 Stampa dei risultati
print("\n📊 RISULTATI TEST DI LEVENE 📊")
print(levene_df)

if not fligner_df.empty:
    print("\n📊 RISULTATI TEST DI FLIGNER-KILLEEN 📊")
    print(fligner_df)

print(f"✅ Risultati salvati in:\n- {levene_path}\n- {fligner_path}")


In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from itertools import combinations

# 📂 Percorsi ai dati
data_path = r"C:\Users\Lorenzo\Desktop\scoring/sleep_characteristics.xlsx"
shapiro_path = r"C:\Users\Lorenzo\Desktop\scoring/shapiro_results.xlsx"
levene_path = r"C:\Users\Lorenzo\Desktop\scoring/levene_results.xlsx"
fligner_path = r"C:\Users\Lorenzo\Desktop\scoring/fligner_results.xlsx"

# 📂 Caricamento dati
df = pd.read_excel(data_path)
shapiro_df = pd.read_excel(shapiro_path)
levene_df = pd.read_excel(levene_path)
fligner_df = pd.read_excel(fligner_path)

# Debug per vedere le colonne effettive
print("Colonne nel file Levene's test:")
print(levene_df.columns.tolist())

# Usando direttamente la colonna come appare nel dataframe
levene_p_column = None
for col in levene_df.columns:
    if "levene" in col.lower() and "p" in col.lower():
        levene_p_column = col
        print(f"Trovata colonna per Levene's test: '{col}'")
        break

# Stessa cosa per Fligner-Killeen
fligner_p_column = None
for col in fligner_df.columns:
    if "fligner" in col.lower() and "p" in col.lower():
        fligner_p_column = col
        print(f"Trovata colonna per Fligner-Killeen test: '{col}'")
        break

# 📌 Definizione gruppi e variabili
group_order = ["CTL", "DNV", "ADV", "DYS"]
numerical_cols = df.columns[2:]  # Escludiamo 'Patient' e 'Group'

# 📂 Creazione directory output
output_dir = r"C:\Users\Lorenzo\Desktop\scoring"
os.makedirs(output_dir, exist_ok=True)

# 📌 DataFrame per salvare i risultati statistici
stat_results = []
posthoc_results = []

# 📊 Stile dei grafici
sns.set_style("whitegrid")

# 📌 Loop sulle variabili numeriche
for col in numerical_cols:
    print(f"\n📊 Analisi per {col}...")

    # **1️⃣ Lettura dei risultati di normalità**
    normal_groups = {}
    for group in group_order:
        p_value_shapiro = shapiro_df[(shapiro_df["Variabile"] == col) & (shapiro_df["Gruppo"] == group)]["Shapiro-Wilk p-value"].values
        normal_groups[group] = p_value_shapiro[0] >= 0.05 if len(p_value_shapiro) > 0 else np.nan

    all_normal = all(normal_groups.values())

    # **2️⃣ Lettura dei test di varianza**
    equal_variance = False  # Default a False

    # Prova a leggere il p-value di Levene se abbiamo trovato la colonna
    if levene_p_column is not None:
        filtered_levene = levene_df[levene_df["Variabile"] == col]
        if not filtered_levene.empty:
            levene_p = filtered_levene[levene_p_column].values
            if len(levene_p) > 0:
                equal_variance = levene_p[0] >= 0.05
                print(f"Levene's test p-value: {levene_p[0]:.5f} → Varianza omogenea: {equal_variance}")

    # Se non abbiamo trovato con Levene, proviamo con Fligner
    elif fligner_p_column is not None:
        filtered_fligner = fligner_df[fligner_df["Variabile"] == col]
        if not filtered_fligner.empty:
            fligner_p = filtered_fligner[fligner_p_column].values
            if len(fligner_p) > 0:
                equal_variance = fligner_p[0] >= 0.05
                print(f"Fligner-Killeen test p-value: {fligner_p[0]:.5f} → Varianza omogenea: {equal_variance}")

    # **3️⃣ Scelta del test statistico**
    if all_normal and equal_variance:
        test_name = "One-Way ANOVA"
        stat_test, p_value = stats.f_oneway(*[df[df["Group"] == g][col].dropna() for g in group_order])
    elif all_normal and not equal_variance:
        test_name = "Welch's ANOVA"
        model = smf.ols(f"{col} ~ C(Group)", data=df).fit()
        p_value = sm.stats.anova_lm(model, typ=2)["PR(>F)"][0]
    else:
        test_name = "Kruskal-Wallis"
        stat_test, p_value = stats.kruskal(*[df[df["Group"] == g][col].dropna() for g in group_order])

    print(f"📊 Test usato: {test_name} → p-value: {p_value:.5f}")

    # **4️⃣ Test post-hoc**
    sig_pairs = []
    if p_value < 0.05:
        print(f"✅ {col}: {test_name} SIGNIFICATIVO (p = {p_value:.4f}) → eseguo test post-hoc.")

        if test_name in ["One-Way ANOVA", "Welch's ANOVA"]:
            tukey_results = pairwise_tukeyhsd(df[col].dropna(), df["Group"].dropna())

            for i, row in enumerate(tukey_results._results_table.data[1:]):
                g1, g2, meandiff, p_val, lower, upper, reject = row
                if p_val < 0.05:
                    sig_pairs.append((g1, g2, p_val))
                    posthoc_results.append({
                        "Variabile": col, "Test": "Tukey HSD",
                        "Gruppo1": g1, "Gruppo2": g2,
                        "p-value": round(p_val, 5), "Significativo": "Sì" if reject else "No"
                    })

    # **5️⃣ Creazione grafico con annotazioni**
    plt.figure(figsize=(8, 6))
    sns.boxplot(x="Group", y=col, hue="Group", data=df, order=group_order, palette="Set2", legend=False)
    plt.title(f"Distribuzione di {col} per gruppo")
    plt.ylabel(col)
    plt.xlabel("Gruppo")
    plt.grid(True, linestyle="--", alpha=0.7)

    if sig_pairs:
        y_max = df[col].max()
        y_min = df[col].min()
        height_step = (y_max - y_min) * 0.1
        pos = y_max + height_step

        for g1, g2, p_val in sig_pairs:
            x1, x2 = group_order.index(g1), group_order.index(g2)
            text = "****" if p_val < 0.0001 else "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*"
            plt.plot([x1, x1, x2, x2], [pos, pos + height_step, pos + height_step, pos], color="black")
            plt.text((x1 + x2) * 0.5, pos + height_step * 1.2, text, ha="center", va="bottom", fontsize=12)
            pos += height_step * 1.5

    plot_path = os.path.join(output_dir, f"{col}_boxplot.png")
    plt.savefig(plot_path, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"📊 Plot salvato in: {plot_path}")


Colonne nel file Levene's test:
['Variabile', 'Levene’s p-value']
Trovata colonna per Levene's test: 'Levene’s p-value'
Trovata colonna per Fligner-Killeen test: 'Fligner-Killeen p-value'

📊 Analisi per Sleep Efficiency (%)...
Levene's test p-value: 0.07199 → Varianza omogenea: True
📊 Test usato: Kruskal-Wallis → p-value: 0.35368
📊 Plot salvato in: C:\Users\Lorenzo\Desktop\scoring\Sleep Efficiency (%)_boxplot.png

📊 Analisi per Total Sleep Time (min)...
Levene's test p-value: 0.05794 → Varianza omogenea: True
📊 Test usato: One-Way ANOVA → p-value: 0.45540
📊 Plot salvato in: C:\Users\Lorenzo\Desktop\scoring\Total Sleep Time (min)_boxplot.png

📊 Analisi per N1%...
Levene's test p-value: 0.37590 → Varianza omogenea: True
📊 Test usato: Kruskal-Wallis → p-value: 0.39741
📊 Plot salvato in: C:\Users\Lorenzo\Desktop\scoring\N1%_boxplot.png

📊 Analisi per N2%...
Levene's test p-value: 0.54135 → Varianza omogenea: True
📊 Test usato: One-Way ANOVA → p-value: 0.41011
📊 Plot salvato in: C:\Users\Lo